## How to use `earthdata-varinfo` to publish UMM-Var entries to CMR

### Version 2.0.0

### Overview:

This version of `earthdata-varinfo` adds functionality to publish records to CMR, along with a single overarching function that wraps the search, download and publication functionality into a single function for the convenience of the end-user. Additionally, the function signatures for `cmr_search.get_granules` and `cmr_search.download_granule` have been updated to accept the full Authorization header, instead of a bearer token, so that they are also compatible with LaunchPad tokens.

### Setting up your environment to run this notebook

```
pip install earthdata-varinfo
```

If this doesn't work, alternatively you can clone the git repository, and install the package in editable mode:

```
git clone https://github.com/nasa/earthdata-varinfo
cd earthdata-varinfo
pip install -e .
```

### Other notebook requirements:

Required packages for `earthdata-varinfo` should be installed as dependencies of PyPI. For local development, without a standard pip installation, third party requirements can be installed from the following files:

```
pip install -r requirements.txt -r dev-requirements.txt
pip install notebook
```

### Current output format
Currently `earthdata-varinfo` produces Python classes that can be used within Python scripts, Jupyter notebooks or a Python Read-Eval-Print Loop (REPL).

`earthdata-varinfo` is also able to create UMM-Var compliant JSON records and publish these records to NASA's Common Metadata Repository (CMR).

### Example usage:
* [M2I1NXASM](https://cmr.uat.earthdata.nasa.gov/search/collections.umm_json?concept-id=C1256535511-EEDTEST)
* [GLDAS_NOAH10_3H](https://cmr.uat.earthdata.nasa.gov/search/collections.umm_json?concept-id=C1256543837-EEDTEST)

In [14]:
# Import CMR environment
from cmr import CMR_UAT

# Import varinfo cmr_search functions
from varinfo.cmr_search import (get_granules, get_granule_link, 
                                download_granule)

# Import VarInfoFromNetCDF4 class and umm_var functions
# for reading and netCDF4 files and creating UMM-Var entries 
from varinfo import VarInfoFromNetCDF4

# Import functions for creating UMM-Var entry and
# publishing a UMM-Var entry
from varinfo.umm_var import (get_all_umm_var, get_umm_var, publish_all_umm_var,
                             publish_umm_var)
# Singular function call
from varinfo.generate_umm_var import generate_collection_umm_var

In [12]:
# Update authorization header to include your bearer token (e.g. 'Bearer TOKEN')
# or LaunchPad token (e.g. TOKEN)
auth_header = 'Header + Token or Only token'

### Get granule response and granule download URL with ```get_granules``` and ```get_granule_link``` for collection **M2I1NXASM**.

In [ ]:
granule_response = get_granules(concept_id='C1256535511-EEDTEST',
                                cmr_env=CMR_UAT,
                                auth_header=auth_header)
link = get_granule_link(granule_response)
link

### Then download the granule locally with `download_granule`

In [ ]:
# Saves granule to current directory or
# out_directory=/usr/given/path
download_granule(link, auth_header=auth_header)

### Instantiate a ```VarInfoFromNetCDF4``` object for a local NetCDF-4 file. 

In [17]:
# Instantiate a VarInfoFromNetCDF4 object for a local NetCDF-4 file.
var_info = VarInfoFromNetCDF4('MERRA2_400.inst1_2d_asm_Nx.20220130.nc4',
                              short_name='M2I1NXASM')

### Retrieve a dictionary of UMM-Var JSON records

In [ ]:
# Keys are the full variable paths, values are 
# UMM-Var schema-compatible, JSON-serialisable dictionaries.
umm_var_dict = get_all_umm_var(var_info)
umm_var_dict

### Publish all the UMM-Var entries for M2I1NXASM

In [ ]:
publish_all_umm_var('C1256535511-EEDTEST',
                    umm_var_dict,
                    auth_header=auth_header,
                    cmr_env=CMR_UAT)

### Publish a single UMM-Var entry

In [ ]:
# Keys are the full variable paths, values are 
# UMM-Var schema-compatible, JSON-serialisable dictionaries.
umm_var_entry = get_umm_var(var_info, var_info.get_variable('/TROPPV'))
umm_var_entry

### Retrieve a dictionary of a single UMM-Var JSON records. 

In [ ]:
publish_umm_var('C1256535511-EEDTEST',
                umm_var_entry,
                auth_header=auth_header,
                cmr_env=CMR_UAT)

### Download a granule for the collection **GLDAS_NOAH10_3H**, generate it's UMM-Var entry and publish it to CMR, with our one function: ```generate_collection_umm_var```

In [ ]:
# Add publish=True if you want publish directly to CMR UAT
generate_collection_umm_var(collection_concept_id='C1256543837-EEDTEST',
                            auth_header=auth_header)